In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from core import CompanyProduct

target = CompanyProduct.same("98point6")

In [3]:
from glassdoor.search import find_review_url

url = find_review_url(target, debug=True)

[SearchResult(title='98point6 Reviews: What Is It Like to Work At 98point6? | Glassdoor', link='https://www.glassdoor.com/Reviews/98point6-Reviews-E1181484.htm', snippet='167 98point6 reviews. A free inside look at company reviews and salaries posted anonymously by employees.', formattedUrl='https://www.glassdoor.com/Reviews/98point6-Reviews-E1181484.htm'), SearchResult(title='98point6 Software Engineer(Internship) Reviews | Glassdoor', link='https://www.glassdoor.com/Reviews/98point6-Reviews-E1181484.htm?filter.jobTitleExact=Software+Engineer(Internship)', snippet='3 98point6 reviews. A free inside look at company reviews and salaries posted anonymously by employees.', formattedUrl='https://www.glassdoor.com/Reviews/98point6-Reviews-E1181484.htm?...')]


In [4]:
from glassdoor.scraper import scrape_reviews, GlassdoorReview

glassdoor_results = await scrape_reviews(url, max_pages=3)
reviews = GlassdoorReview.parse_reviews(glassdoor_results)


2024-07-31 16:10:51.899 | INFO     | glassdoor.scraper:scrape_reviews:110 - scraping reviews from https://www.glassdoor.com/Reviews/98point6-Reviews-E1181484.htm
2024-07-31 16:11:46.975 | INFO     | glassdoor.scraper:scrape_reviews:118 - scraped first page of reviews of https://www.glassdoor.com/Reviews/98point6-Reviews-E1181484.htm, scraping remaining 2 pages
2024-07-31 16:12:08.032 | INFO     | glassdoor.scraper:scrape_reviews:128 - scraped 30 reviews from https://www.glassdoor.com/Reviews/98point6-Reviews-E1181484.htm in 3 pages


In [5]:
from glassdoor.summarizer import summarize

summary = summarize(target, reviews)
print(summary.content)

The prompt context has 13,916 characters in 30 reviews
# Summary of 98point6 Glassdoor Reviews

## Key Aspects

### Leadership
**Pros:**
- Some reviews highlight strong leadership that cares about employees and takes action on feedback (e.g., "Strong leadership team that cares about employees" (Anonymous on 2023-01-20)).
  
**Cons:**
- Many reviews criticize leadership for being tone-deaf, inexperienced, and lacking emotional intelligence. There are mentions of a "boys' club" mentality and ineffective management (e.g., "Ineffective clinical leadership with boys' club attitude" (Physician on 2022-04-07)).
- Frequent changes in leadership and strategy have led to instability and low morale (e.g., "Exec team doesn't seem to have long term strategy" (Software Engineer on 2024-01-17)).

### Compensation and Benefits
**Pros:**
- Benefits, including PTO, are generally viewed positively (e.g., "PTO and time-off are better than most companies" (Anonymous on 2022-03-13)).
  
**Cons:**
- Compensa